In [1]:
import selenium
from selenium import webdriver as wb
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import os
import re
import shutil
import glob
from random import randrange
from tqdm import tqdm
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import config
import sys

In [2]:
options = Options()
options = wb.ChromeOptions()
download_folder = "C:\\Users\\Bhagya\\Resume Analyzer\\ACTUAL PROJECT\\resumes"
prefs = {
        "download.default_directory": download_folder,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.disable_download_protection": True,
        "safebrowsing.enabled": True
        }
options.add_experimental_option("prefs", prefs)
options.add_argument("window-size=1200x600")
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
driver = wb.Chrome(options=options)

In [3]:
driver.get('https://www.linkedin.com/feed')
clickSignin = driver.find_element_by_xpath('/html/body/div/main/p/a').click()

In [5]:
username = driver.find_element_by_id('username')
username.send_keys(config.username)
passwd = driver.find_element_by_id('password')
passwd.send_keys(config.password)
passwd.send_keys(Keys.ENTER)
time.sleep(2)

In [6]:
s = driver.find_element_by_class_name('nav-search-typeahead')
sr = s.find_element_by_class_name('search-global-typeahead__input')
sr.send_keys('Data Scientist')
sr.send_keys(Keys.ENTER)
time.sleep(2)

In [7]:
## trick:search by div class only
ao = driver.find_element_by_class_name('authentication-outlet')
ng = ao.find_element_by_class_name('neptune-grid')
sf = ng.find_element_by_class_name('search-filters-bar') 
people = sf.find_element_by_class_name('search-vertical-filter__filter-item').click()
time.sleep(2)

In [10]:
l = driver.find_element_by_class_name('peek-carousel')
l.find_element_by_class_name('search-s-facet--geoRegion').click()
to = driver.find_element_by_xpath("//input[@role='combobox'][@placeholder='Add a country/region']") # changed from l to driver
to.clear
to.send_keys('Orange County, California Area')
time.sleep(2)
to.send_keys(Keys.DOWN, Keys.RETURN)
driver.find_elements_by_tag_name('button')[11].click()

In [11]:
page_urls = []
initial_url = driver.current_url
page_urls.append(initial_url)
for i in range(2,16):
    url = initial_url+"&page=" + str(i)
    page_urls.append(url)

In [13]:
listOflinks = []
for i in tqdm(page_urls):
    driver.get(i)
    results = driver.find_elements_by_class_name("search-result__occluded-item")
    for result in results:
        hover = ActionChains(driver).move_to_element(result)
        hover.perform()
        time.sleep(randrange(3,6))
        links = result.find_element_by_class_name('search-result__result-link').get_property('href')
        listOflinks.append(links)
        time.sleep(randrange(3,6))


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [21:57<00:00, 88.53s/it]

In [14]:
len(listOflinks)

150

In [15]:
def download_wait(directory, timeout, nfiles=None):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds to wait until timing out.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in directory:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

In [ ]:
for i in tqdm(listOflinks):
    driver.get(i)
    m = driver.find_element_by_class_name("flex-1")
    m.find_element_by_class_name("pv-s-profile-actions__overflow").click()
    p_name = m.find_element_by_xpath('/html/body/div[5]/div[3]/div[3]/div/div/div/div/div[2]/main/div[1]/section/div[2]/div[2]/div[1]/ul[1]/li[1]').text
    name = p_name.split(',')[0]
    m.find_element_by_class_name('pv-s-profile-actions--save-to-pdf').click()
    time.sleep(randrange(5, 10))
    filename = max(glob.iglob(download_folder +'/*'), key=os.path.getmtime)
    shutil.move(filename,os.path.join(download_folder, name + '.pdf'))
    #download_wait(download_folder, 20)
    time.sleep(randrange(5, 10))
    print(name)


  0%|                                                                                          | 0/150 [00:00<?, ?it/s]

Farah Farouk



  1%|▌                                                                                 | 1/150 [00:21<52:51, 21.28s/it]

Shantanu Mishra



  1%|█                                                                                 | 2/150 [00:44<54:11, 21.97s/it]

...


 44%|███████████████████████████████████▋                                             | 66/150 [23:36<31:11, 22.28s/it]

James Fitton



 45%|████████████████████████████████████▏                                            | 67/150 [23:51<27:51, 20.14s/it]

Udit Narula



 45%|████████████████████████████████████▋                                            | 68/150 [24:06<25:20, 18.54s/it]

In [ ]:
driver.close()

In [ ]:
import sys
def role(self,data):
    print(data[sys.argv[0]])
    return data